<h2>Digital History: The Story So Far</h2>

As the field of Digital History continues to grow, so too does the number of tools, software, and coding packages built to support and advance digital history in practice. The range of this is at times staggering: from applications suitable for the most novice of digital historians, to coding guides and tools for those working to more nuanced and specific end-goals, researchers have an ability to engage with their materials in digital, quantitative ways on a never before seen level. Often we focus primarily on the new *findings* that come of out this new way of approaching research - but what about the ways we get to those findings?

Regardless of the type of digital analysis being performed or even the software being used, the process is normally the same: input some data, click some buttons or run some code (perhaps a couple of times over to edit the code and adjust the outcomes), and get your end result.

You've got an outcome - but do you know how you've got from a to b? It's likely that variables have been written over several times along the way, and the data has changed from one type to another, been filtered or added to, and decision after decision has been made without necessarily knowing it. Each little adjustment or re-run of the code has contributed to the research process and is critical to the end output or findings.

But how do we keep track?

<h1>Hello <i>kiara</i>.</h1>
<h3>Introducing <i>kiara</i>, a new data orchestration tool.</h3>

This new tool incorporates a number of different digital research approaches, but most importantly documents and encourages users to critically reflect on the process and use of DH tools. In doing so, the software opens up the black box of digital research, moving away from button-clicking software and making digital research more transparent and open to commentary, replicability, and criticism. It not only makes the research process itself more open, allowing users to visualise and examine the individual steps from start to finish, but also allows them to track changes to the data itself, something that is either imperceptible or, perhaps more importantly, *forgotten about* in traditional digital history methods and tools. *kiara* therefore acts as a 'wrapper' to this digital reserach process, tracking and documenting the steps and changes to the data, producing a veritable map of the journey that can be reflected upon and shared.

This tutorial will walk you through installation of *kiara* in Jupyter Notebooks, and some basic but essential functions that can be built on in further notebooks. At the end, it will showcase the **data lineage**, having tracked the research process and changes to the data from start to finish.

This tutorial requires you to know **python** and **SQL**.

<h2>Installation</h2>

First, we need to check if *kiara* and its plugins are already installed, and install them if not. There are currently a number of plugins, including:

- <a href="https://dharpa.org/kiara_plugin.core_types/latest/">`kiara_plugin.core-types`</a>
- <a href="https://dharpa.org/kiara_plugin.onboarding/latest/">`kiara_plugin.onboarding`</a>
- <a href="https://dharpa.org/kiara_plugin.tabular/latest/">`kiara_plugin.tabular`</a>
- <a href="https://dharpa.org/kiara_plugin.network_analysis/latest/">`kiara_plugin.network_analysis`</a>
- <a href="https://dharpa.org/kiara_plugin.language_processing/latest/">`kiara_plugin.language_processing`</a>
- <a href="https://dharpa.org/kiara_plugin.html/latest/">`kiara_plugin.html`</a>
- <a href="https://dharpa.org/kiara_plugin.streamlit/latest/">`kiara_plugin.streamlit`</a>

All of these will be installed automatically alongside *kiara*, using the code below:

In [1]:
try:
    from kiara_plugin.jupyter import ensure_kiara_plugins
except:
    import sys
    print("Installing 'kiara_plugin.jupyter'...")
    !{sys.executable} -m pip install -q kiara_plugin.jupyter
    from kiara_plugin.jupyter import ensure_kiara_plugins

ensure_kiara_plugins()


<h2>Running <i>kiara</i></h2>

In order to use *kiara*, we need to create a `KiaraAPI` instance. An API allows us to control and interact with *kiara* and its functions. In *kiara* this also allows us to get more information about what can be done (and what is happening) to our data as we go. For more on what can be done with the API, see the *kiara* API documentation <a href = "https://dharpa.org/kiara/latest/reference/kiara/interfaces/python_api/__init__/#kiara.interfaces.python_api.KiaraAPI">here</a>.

In [2]:
from kiara.api import KiaraAPI

kiara = KiaraAPI.instance()

Now we have an API in place, we can get more information about what we can do in *kiara*. Let's start by asking *kiara* to list all the operations that are included with the plugins we just installed.

In [3]:
kiara.list_operation_ids()

['assemble.network_data.from.files',
 'assemble.network_data.from.tables',
 'compute.modularity_group',
 'create.betweenness_rank_list',
 'create.closeness_rank_list',
 'create.cut_point_list',
 'create.database.from.file',
 'create.database.from.file_bundle',
 'create.database.from.table',
 'create.degree_rank_list',
 'create.eigenvector_rank_list',
 'create.network_data.from.file',
 'create.stopwords_list',
 'create.table.from.file',
 'create.table.from.file_bundle',
 'date.check_range',
 'date.extract_from_string',
 'download.file',
 'download.file_bundle',
 'export.file.as.file',
 'export.network_data.as.csv_files',
 'export.network_data.as.graphml_file',
 'export.network_data.as.sql_dump',
 'export.network_data.as.sqlite_db',
 'export.table.as.csv_file',
 'extract.date_array.from.table',
 'file_bundle.pick.file',
 'file_bundle.pick.sub_folder',
 'generate.LDA.for.tokens_array',
 'get_lineage_data',
 'import.database.from.local_file_path',
 'import.file',
 'import.file_bundle',
 'i

<h3>Downloading Files</h3>

Great, now we know the different kind of operations we can use with *kiara*. Let's start by introducing some files to our notebook, using the `download.file` function.<br/>
First we want to find out what this operation does, and just as importantly, what inputs it needs to work.

In [4]:
kiara.retrieve_operation_info('download.file')

Author(s)                                                                                                                                  
                     Markus Binsteiner   markus@frkl.io                                                                                     
                                                                                                                                            
 Context                                                                                                                                    
                     Tags         onboarding                                                                                                
                     Labels       package: kiara_plugin.onboarding                                                                          
                     References   source_repo: ]8;id=40535;https://github.com/DHARPA-Project/kiara_plugin.onboarding\https://github.com/DHARPA-Project/kiara_plugin.onboarding]8;;\                                    
                                  documentation: ]8;id=185725;https://DHARPA-Project.github.io/kiara_plugin.onboarding/\https://DHARPA-Project.github.io/kiara_plugin.onboarding/]8;;\                                  
                                                                                                                                            
 Operation details                                                                                                                          
                     Documentation   Download a single file from a remote location.                                                         
                                                                                                                                            
                                     The result of this operation is a single value of type 'file' (basically an array of raw bytes),       
                                     which can then be used in other modules to                                                             
                                     create more meaningful data structures.                                                                
                                                                                                                                            
                     Inputs                                                                                                                 
                                       field name   type     description                                    Required   Default              
                                      ──────────────────────────────────────────────────────────────────────────────────────────────────    
                                       url          string   The url of the file to download.               yes        -- no default --     
                                       file_name    string   The file name to use for the downloaded        no         -- no default --     
                                                             file.                                                                          
                                                                                                                                            
                                                                                                                                            
                     Outputs                                                                                                                
                                       field name          type   description                                                               
                                      ──────────────────────────────────────────────────────────────────────────────────────────────────    
                                   

So from this, we know that `download.file` will download a single file from a remote location for us to use in *kiara*. <br/>
We need to give the function a **url** and, if we want, a **file name**. These are the <span style="color:green">**inputs**</span>. <br/>
In return, we will get the **file** and **metadata** about the file as our <span style="color:red">**outputs**</span>.

Let's give this a go using some *kiara* sample data.

First we define our <span style="color:green">inputs</span>, then use `kiara.run_job` with our chosen operation, `download.file`, and save this as our <span style="color:red">outputs</span>.

In [5]:
inputs = {
        "url": "https://raw.githubusercontent.com/DHARPA-Project/kiara.examples/main/examples/data/network_analysis/journals/JournalNodes1902.csv",
        "file_name": "JournalNodes1902.csv"
}

outputs = kiara.run_job('download.file', inputs=inputs)

Let's print out our <span style="color:red">outputs</span> and see what that looks like.

In [6]:
outputs

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                          │
│   field               value                                                                                                              │
│  ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────  │
│   download_metadata                                                                                                                      │
│                         dict data     {                                                                                                  │
│                                         "response_headers": [                                                                            │
│                                           {                                                                                              │
│                                             "connection": "keep-alive",                                                                  │
│                                             "content-length": "7436",                                                                    │
│                                             "cache-control": "max-age=300",                                                              │
│                                             "content-security-policy": "default-src 'none'; style-src 'unsafe-inline'; sandbox",         │
│                                             "content-type": "text/plain; charset=utf-8",                                                 │
│                                             "etag": "W/\"641ae85d69e5836d27ea8906aba0a33b48b0f3ed0ed4c40d21a07fccebdd238d\"",            │
│                                             "strict-transport-security": "max-age=31536000",                                             │
│                                             "x-content-type-options": "nosniff",                                                         │
│                                             "x-frame-options": "deny",                                                                   │
│                                             "x-xss-protection": "1; mode=block",                                                         │
│                                             "x-github-request-id": "4B08:14F1:1D20A3C:1E64A7E:6461FB12",                                 │
│                                             "content-encoding": "gzip",                                                                  │
│                                             "accept-ranges": "bytes",                                                                    │
│                                             "date": "Mon, 15 May 2023 11:27:55 GMT",                                                     │
│                                             "via": "1.1 varnish",                                                                        │
│                                             "x-served-by": "cache-fra-eddf8230042-FRA",                                                  │
│                                             "x-cache": "HIT",                                                                            │
│                                             "x-cache-hits": "1",                                                                         │
│                                             "x-timer": "S1684150075.074095,VS0,VE150",                                                   │
│                                             "vary": "Authorization,Accept-Encoding,Origin",                                              │
│                                             "acces

Great! We've successfully downloaded the file, and we can see there's lots of information here.

At the moment, we're most interested in the **file** output. This contains the actual *contents* of the file that we have just downloaded.

Let's separate this out and store it in a separate variable for us to use.

In [7]:
downloaded_file = outputs['file']

<h3>New Formats: Creating and Converting</h3>

What next? We could transform the downloaded file contents into a different format. <br/>
Let's use the operation list earlier, and look for something that allows us to create something out of our new file.

In [8]:
kiara.list_operation_ids('create')

['create.betweenness_rank_list',
 'create.closeness_rank_list',
 'create.cut_point_list',
 'create.database.from.file',
 'create.database.from.file_bundle',
 'create.database.from.table',
 'create.degree_rank_list',
 'create.eigenvector_rank_list',
 'create.network_data.from.file',
 'create.stopwords_list',
 'create.table.from.file',
 'create.table.from.file_bundle']

Our file was orginally in a CSV format, so let's make a table using `create.table.from.file`. 

Just like when we used `download.file`, we can double check what this does, and what <span style="color:green">inputs</span> and <span style="color:red">outputs</span> this involves.

This time, we're also going to use a variable to store the operation in - this is especially handy if the operation has a long name, or if you want to use the same operation more than once without retyping it.

In [9]:
op_id = 'create.table.from.file'

kiara.retrieve_operation_info(op_id)

Author(s)                                                                                                                                  
                     Markus Binsteiner   markus@frkl.io                                                                                     
                                                                                                                                            
 Context                                                                                                                                    
                     Tags         tabular                                                                                                   
                     Labels       package: kiara_plugin.tabular                                                                             
                     References   source_repo: ]8;id=901826;https://github.com/DHARPA-Project/kiara_plugin.tabular\https://github.com/DHARPA-Project/kiara_plugin.tabular]8;;\                                       
                                  documentation: ]8;id=57780;https://DHARPA-Project.github.io/kiara_plugin.tabular/\https://DHARPA-Project.github.io/kiara_plugin.tabular/]8;;\                                     
                                                                                                                                            
 Operation details                                                                                                                          
                     Documentation   Create a table from a file, trying to auto-determine the format of said file.                          
                                                                                                                                            
                     Inputs                                                                                                                 
                                       field name            type      description                          Required   Default              
                                      ──────────────────────────────────────────────────────────────────────────────────────────────────    
                                       file                  file      The source value (of type 'file').   yes        -- no default --     
                                       first_row_is_header   boolean   Whether the first row of the file    no         -- no default --     
                                                                       is a header row. If not provided,                                    
                                                                       kiara will try to auto-determine.                                    
                                                                                                                                            
                                                                                                                                            
                     Outputs                                                                                                                
                                       field name   type    description                                                                     
                                      ──────────────────────────────────────────────────────────────────────────────────────────────────    
                                       table        table   The result value (of type 'table').                                             
                                                                                                                                            
                  

Great, we have all the information we need now.

Let's go again.

First we define our <span style="color:green">inputs</span>, that is the downloaded file we saved earlier as well as telling kiara that the first row should be read as a header.

Then use `kiara.run_job` with our chosen operation, this time stored as `op_id`.

Once this is saved as our <span style="color:red">outputs</span>, we can print it out.

In [10]:
inputs = {
    "file": downloaded_file,
    "first_row_is_header": True
}

outputs = kiara.run_job(op_id, inputs=inputs)

outputs

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                          │
│   field   value                                                                                                                          │
│  ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────  │
│   table                                                                                                                                  │
│             Id    Label            JournalType      City        CountryNetworkT   PresentDayCoun   Latitude    Longitude    Language     │
│            ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────    │
│             75    Psychiatrische   specialized: p   Amsterdam   Netherlands       Netherlands      52.366667   4.9          Dutch        │
│             36    The American J   specialized: p   Baltimore   United States     United States    39.289444   -76.615278   English      │
│             208   The American J   specialized: p   Baltimore   United States     United States    39.289444   -76.615278   English      │
│             295   Die Krankenpfl   specialized: t   Berlin      German Empire     Germany          52.52       13.405       German       │
│             296   Die deutsche K   general medici   Berlin      German Empire     Germany          52.52       13.405       German       │
│             300   Therapeutische   specialized: t   Berlin      German Empire     Germany          52.52       13.405       German       │
│             1     Allgemeine Zei   specialized: p   Berlin      German Empire     Germany          52.52       13.405       German       │
│             7     Archiv für Psy   specialized: p   Berlin      German Empire     Germany          52.52       13.405       German       │
│             10    Berliner klini   general medici   Berlin      German Empire     Germany          52.52       13.405       German       │
│             13    Charité Annale   general medici   Berlin      German Empire     Germany          52.52       13.405       German       │
│             21    Monatsschrift    specialized: p   Berlin      German Empire     Germany          52.52       13.405       German       │
│             29    Virchows Archi   specialized: a   Berlin      German Empire     Germany          52.52       13.405       German       │
│             31    Zeitschrift fü   specialized: p   Berlin      German Empire     Germany          52.52       13.405       German       │
│             42    Vierteljahrssc   specialized: a   Berlin      German Empire     Germany          52.52       13.405       German       │
│             47    Centralblatt f   specialized: p   Berlin      German Empire     Germany          52.52       13.405       German       │
│             50    Russische medi   general medici   Berlin      German Empire     Germany          52.52       13.405       German       │
│             ...   ...              ...              ...         ...               ...              ...         ...          ...          │
│             ...   ...              ...              ...         ...               ...              ...         ...          ...          │
│             277   L'arte medica    general medici   Turin       Italy             Italy            45.079167   7.676111     Italian      │
│             288   Allgemeine öst   specialized: a   Vienna      Austro-Hungaria   Austria          48.2        16.366667    German       │
│             18    Jahrbücher für   specialized: p   Vienna      Austro-Hungaria   Austria          48.2        16.366667    German       │
│             30    Wiener klinisc   general medic

This has done exactly what we wanted, and shown the contents from the downloaded file as a table. But we are also interested in some general (mostly internal) information and metadata, this time for the new table we have just created, rather than the original file itself.

Let's have a look.

In [11]:
outputs_table = outputs['table']

outputs_table

 value_id            6a44d2e8-d291-4324-aee2-eeec6a9c34b3                                                                                  
  kiara_id            b7f21777-b559-4a57-82e0-05999eed91e2                                                                                  
                                                                                                                                            
                      ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────  
  data_type_info                                                                                                                            
                        data_type_name     table                                                                                            
                        data_type_config   {}                                                                                               
                        characteristics    {                                                                                                
                                             "is_scalar": false,                                                                            
                                             "is_json_serializable": false                                                                  
                                           }                                                                                                
                        data_type_class                                                                                                     
                                             python_class_name    TableType                                                                 
                                             python_module_name   kiara_plugin.tabular.data_types.table                                     
                                             full_name            kiara_plugin.tabular.data_types.table.TableType                           
                                                                                                                                            
                                                                                                                                            
  destiny_backlinks   {}                                                                                                                    
  enviroments         None                                                                                                                  
  property_links      {                                                                                                                     
                        "metadata.python_class": "fc9be60a-2d5f-4265-b4e4-081f4e521ec3",                                                    
                        "metadata.table": "45e12aec-ea47-4118-ac21-b5ad49672ce9"                                                            
                      }                                                                                                                     
  value_hash          zdpuAn89Et1ENzfoASJRYcWEceyfRiPg664mN4nnHLFnjRLyg                                                                     
  value_schema                                                                                                                              
                        type          table                                                                                                 
                        type_config   {}                                                                                                    
                        default       __not_set__                                                                                           
                        optional      False          

<h3>Querying our Data</h3>

So now we have downloaded our file and converted it into a table, we want to actually explore it.

To do this, we can query the table using **SQL** and some functions already included in *kiara*.

Let's take another look at that operation list, this time looking for functions that let us 'query'.


In [12]:
kiara.list_operation_ids('query')

['query.database', 'query.table']

Well, we already know our file has been converted into a table, so let's have a look at `query.table`.

In [13]:
kiara.retrieve_operation_info('query.table')

Author(s)                                                                                                                                  
                     Markus Binsteiner   markus@frkl.io                                                                                     
                                                                                                                                            
 Context                                                                                                                                    
                     Tags         tabular                                                                                                   
                     Labels       package: kiara_plugin.tabular                                                                             
                     References   source_repo: ]8;id=895;https://github.com/DHARPA-Project/kiara_plugin.tabular\https://github.com/DHARPA-Project/kiara_plugin.tabular]8;;\                                       
                                  documentation: ]8;id=363627;https://DHARPA-Project.github.io/kiara_plugin.tabular/\https://DHARPA-Project.github.io/kiara_plugin.tabular/]8;;\                                     
                                                                                                                                            
 Operation details                                                                                                                          
                     Documentation   Execute a sql query against an (Arrow) table.                                                          
                                                                                                                                            
                                     The default relation name for the sql query is 'data', but can be modified by the 'relation_name'      
                                     config option/input.                                                                                   
                                                                                                                                            
                                     If the 'query' module config option is not set, users can provide their own query, otherwise the       
                                     pre-set                                                                                                
                                     one will be used.                                                                                      
                                                                                                                                            
                     Inputs                                                                                                                 
                                       field name      type     description                                 Required   Default              
                                      ──────────────────────────────────────────────────────────────────────────────────────────────────    
                                       table           table    The table to query                          yes        -- no default --     
                                       query           string   The query, use the value of the             yes        -- no default --     
                                                                'relation_name' input as table, e.g.                                        
                                                                'select * from data'.                                                       
                                       relation_name   string   The name the table is referred to in the    no         data                 
                                           

So from this information, we only need to provide the **table** itself, and our **query**.

Let's work out how many of these journals were published in Berlin.

In [14]:
inputs = {
    "table": outputs_table,
    "query": "SELECT * from data where City like 'Berlin'"
}

outputs = kiara.run_job('query.table', inputs=inputs)

outputs

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                          │
│   field          value                                                                                                                   │
│  ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────  │
│   query_result                                                                                                                           │
│                    Id    Label            JournalType      City     CountryNetwor   PresentDayCoun   Latitude   Longitude   Language     │
│                   ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────    │
│                    295   Die Krankenpfl   specialized: t   Berlin   German Empire   Germany          52.52      13.405      German       │
│                    296   Die deutsche K   general medici   Berlin   German Empire   Germany          52.52      13.405      German       │
│                    300   Therapeutische   specialized: t   Berlin   German Empire   Germany          52.52      13.405      German       │
│                    1     Allgemeine Zei   specialized: p   Berlin   German Empire   Germany          52.52      13.405      German       │
│                    7     Archiv für Psy   specialized: p   Berlin   German Empire   Germany          52.52      13.405      German       │
│                    10    Berliner klini   general medici   Berlin   German Empire   Germany          52.52      13.405      German       │
│                    13    Charité Annale   general medici   Berlin   German Empire   Germany          52.52      13.405      German       │
│                    21    Monatsschrift    specialized: p   Berlin   German Empire   Germany          52.52      13.405      German       │
│                    29    Virchows Archi   specialized: a   Berlin   German Empire   Germany          52.52      13.405      German       │
│                    31    Zeitschrift fü   specialized: p   Berlin   German Empire   Germany          52.52      13.405      German       │
│                    42    Vierteljahrssc   specialized: a   Berlin   German Empire   Germany          52.52      13.405      German       │
│                    47    Centralblatt f   specialized: p   Berlin   German Empire   Germany          52.52      13.405      German       │
│                    50    Russische medi   general medici   Berlin   German Empire   Germany          52.52      13.405      German       │
│                    76    Deutsche Aerzt   general medici   Berlin   German Empire   Germany          52.52      13.405      German       │
│                    87    Monatsschrift    specialized: g   Berlin   German Empire   Germany          52.52      13.405      German       │
│                    108   Archiv für kli   specialized: s   Berlin   German Empire   Germany          52.52      13.405      German       │
│                    113   Zeitschrift fü   general medici   Berlin   German Empire   Germany          52.52      13.405      German       │
│                    159   Deutsche milit   specialized: m   Berlin   German Empire   Germany          52.52      13.405      German       │
│                    162   Jahresbericht    specialized: p   Berlin   German Empire   Germany          52.52      13.405      German       │
│                    192   Ärztliche Sach   general medici   Berlin   German Empire   Germany          52.52      13.405      German       │
│                    198   Zeitschrift fü   specialized: p   Berlin   German Empire   Germany          52.52      13.405      German       │
│                    258   Der Pfarrbote    news med

The function has returned the table with just the results we were looking for from the SQL query. 

Let's narrow this further, and find all the journals that are just about general medicine and published in Berlin.

We can re-use the `query.table` function and the table we've just made, stored in `outputs['query_result']`

In [15]:
inputs = {
    "table" : outputs['query_result'],
    "query" : "SELECT * from data where JournalType like 'general medicine'"
}

outputs = kiara.run_job('query.table', inputs=inputs)

outputs

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                          │
│   field          value                                                                                                                   │
│  ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────  │
│   query_result                                                                                                                           │
│                    Id    Label            JournalType      City     CountryNetwork   PresentDayCou   Latitude   Longitude   Language     │
│                   ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────    │
│                    296   Die deutsche K   general medici   Berlin   German Empire    Germany         52.52      13.405      German       │
│                    10    Berliner klini   general medici   Berlin   German Empire    Germany         52.52      13.405      German       │
│                    13    Charité Annale   general medici   Berlin   German Empire    Germany         52.52      13.405      German       │
│                    50    Russische medi   general medici   Berlin   German Empire    Germany         52.52      13.405      German       │
│                    76    Deutsche Aerzt   general medici   Berlin   German Empire    Germany         52.52      13.405      German       │
│                    113   Zeitschrift fü   general medici   Berlin   German Empire    Germany         52.52      13.405      German       │
│                    192   Ärztliche Sach   general medici   Berlin   German Empire    Germany         52.52      13.405      German       │
│                                                                                                                                          │
│                                                                                                                                          │
╰──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

<h3>Recording and Tracing our Data</h3>

We've quite a few changes to this table, so let's double check the information about this new table we've created with our queries.

In [16]:
query_output = outputs['query_result']

query_output

 value_id            93ece2e6-6f73-40df-a6a2-643bb1069932                                                                                  
  kiara_id            b7f21777-b559-4a57-82e0-05999eed91e2                                                                                  
                                                                                                                                            
                      ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────  
  data_type_info                                                                                                                            
                        data_type_name     table                                                                                            
                        data_type_config   {}                                                                                               
                        characteristics    {                                                                                                
                                             "is_scalar": false,                                                                            
                                             "is_json_serializable": false                                                                  
                                           }                                                                                                
                        data_type_class                                                                                                     
                                             python_class_name    TableType                                                                 
                                             python_module_name   kiara_plugin.tabular.data_types.table                                     
                                             full_name            kiara_plugin.tabular.data_types.table.TableType                           
                                                                                                                                            
                                                                                                                                            
  destiny_backlinks   {}                                                                                                                    
  enviroments         None                                                                                                                  
  property_links      {                                                                                                                     
                        "metadata.python_class": "3548c15b-9973-4d5b-973b-f9791f7f0050",                                                    
                        "metadata.table": "14fe502b-959c-4044-aa1d-f23852f49925"                                                            
                      }                                                                                                                     
  value_hash          zdpuB2LfZYHdiuR1sxy2ZkjPZ7JDnhysN48Y4RN9WNT4AvNN6                                                                     
  value_schema                                                                                                                              
                        type          table                                                                                                 
                        type_config   {}                                                                                                    
                        default       __not_set__                                                                                           
                        optional      False          

Looks good!

We might have changed things around, but we can still get lots of information about all our data.

More importantly, *kiara* is able to trace all of these changes, tracking the inputs and outputs and giving them all different identifiers, so you know exactly what has happened to your data. <br/>Check it out!

In [17]:
query_output.lineage

query.table
├── input: query (string) = 524d4801-e49e-40fc-8482-912c6c300b2f
├── input: relation_name (string) = 4972e984-549a-41b4-9666-bd43ab7cf354
└── input: table (table) = be313c7c-f89b-41da-be83-2991326189ea
    └── query.table
        ├── input: query (string) = acf54041-dc49-40e8-ad47-5fbfd1a740ff
        ├── input: relation_name (string) = 4972e984-549a-41b4-9666-bd43ab7cf354
        └── input: table (table) = 6a44d2e8-d291-4324-aee2-eeec6a9c34b3
            └── create.table
                ├── input: file (file) = 87c80285-26d6-460e-b834-5d8c8c37672f
                │   └── download.file
                │       ├── input: file_name (string) = 1e6cd0aa-d127-4cc5-a8a2-906d076ba2ea
                │       └── input: url (string) = a91a5efe-a0f8-4405-9321-f33a61e3a4e1
                └── input: first_row_is_header (boolean) = 4c0f60f3-d8d4-4511-87c3-c2f0acbfcb49

In [18]:
kiara.retrieve_operation_info('get_lineage_data')

Author(s)                                                                                                                                  
                     Mariella De Crouy   mariella.decrouychanel@uni.lu                                                                      
                     Markus Binsteiner   markus@frkl.io                                                                                     
                                                                                                                                            
 Context                                                                                                                                    
                     Tags         dh_tagung_2023                                                                                            
                     Labels       package: kiara_plugin.dh_tagung_2023                                                                      
                     References   source_repo: ]8;id=63228;https://github.com/DHARPA-Project/kiara_plugin.dh_tagung_2023\https://github.com/DHARPA-Project/kiara_plugin.dh_tagung_2023]8;;\                                
                                  documentation: ]8;id=778210;https://DHARPA-Project.github.io/kiara_plugin.dh_tagung_2023/\https://DHARPA-Project.github.io/kiara_plugin.dh_tagung_2023/]8;;\                              
                                                                                                                                            
 Operation details                                                                                                                          
                     Documentation   Get lineage data to display in visualization.                                                          
                                                                                                                                            
                     Inputs                                                                                                                 
                                       field name   type    description                                     Required   Default              
                                      ──────────────────────────────────────────────────────────────────────────────────────────────────    
                                       table        table   The table for which we need lineage data.       yes        -- no default --     
                                                                                                                                            
                                                                                                                                            
                     Outputs                                                                                                                
                                       field name     type   description                                                                    
                                      ──────────────────────────────────────────────────────────────────────────────────────────────────    
                                       lineage_dict   dict   The dict containing lineage data.                                              
                                                                                                                                            
                  

Prototype-stage process to add the lineage visualization
Playground is needed as a dependency

In [ ]:
inputs = {
    "table" : query_output
}

outputs = kiara.run_job('get_lineage_data', inputs=inputs)

Display lineage visualization

In [ ]:
%%capture
! pip install observable_jupyter

In [ ]:
from observable_jupyter import embed

In [ ]:
embed('@dharpa-project/kiara-data-lineage', cells=['displayViz', 'style'], inputs={'dataset':outputs['lineage_dict'].data.dict_data})

Even though we are only actually asking for the **data lineage** using the *last* SQL query and the table it made, *kiara* shows us everything that has happened since we first downloaded the file. This helps us keep an eye on the research process *and* the changes we are making to the data at the same time!

<h2>What next...?</h2>

That's great, you've completed the first notebook and successfully installed *kiara*, downloaded files, tested out some functions, and are able to see what this does to your data. 

Now you can check out the other plugin packages to explore how this helps you manage and trace your data while using digital analysis tools!
